**Importing Labraries**

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Load Dataset**

In [3]:
company_master=pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - SKU Master.xlsx')
company_master.head()



,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [4]:
company_report=pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Order Report.xlsx')
company_report.head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [5]:
company_rep_mast=pd.merge(company_report,company_master, how='outer')


In [6]:
company_rep_mast['total_wt (KG)']=(company_rep_mast['Order Qty']*company_rep_mast['Weight (g)'])/1000
company_rep_master=company_rep_mast.groupby('ExternOrderNo').sum()
company_rep_master.sort_values(by=['ExternOrderNo'],inplace=True)
company_rep_master.reset_index(inplace = True)
company_rep_master.rename(columns={'ExternOrderNo':'Order ID'},inplace=True)
company_rep_master

,Order ID,Order Qty,Weight (g),total_wt (KG)
0,2001806210,2.0,220,0.220
1,2001806226,2.0,240,0.480
2,2001806229,3.0,500,0.500
3,2001806232,8.0,377,1.302
4,2001806233,2.0,245,0.245
...,...,...,...,...
119,2001821995,2.0,477,0.477
120,2001822466,10.0,352,1.376
121,2001823564,6.0,336,0.672
122,2001825261,13.0,611,1.557


In [7]:
company_pin=pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Pincode Zones.xlsx')
company_pin

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [8]:
cou_rates=pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Rates.xlsx')
cou_rates

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [9]:
cou_invoice=pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Invoice.xlsx')



In [10]:
compare=pd.merge(cou_invoice,company_pin,on='Customer Pincode')
compare.drop_duplicates(subset="Order ID", inplace=True)



In [11]:
compare.sort_values(by=['Order ID'],inplace=True)
compare.reset_index(drop=True,inplace=True)
compare

,AWB Code,Order ID,Charged Weight,Warehouse Pincode_x,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Warehouse Pincode_y,Zone_y
0,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,121003,b
1,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,121003,d
2,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,121003,d
3,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,121003,d
4,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,121003,b
...,...,...,...,...,...,...,...,...,...,...
119,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,121003,b
120,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,121003,b
121,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,121003,d
122,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,121003,d


**Creating resultant dataframe**

In [12]:
orderid=compare['Order ID']
awb=compare['AWB Code']
total_wt=company_rep_master['total_wt (KG)']


In [13]:
def round_up_to_nearest_half_int(num):
    return math.ceil(num*2)/2

In [14]:
wt_slab_x=total_wt.apply(round_up_to_nearest_half_int)
toatl_wt_cou=compare['Charged Weight']
wt_slab_cou=toatl_wt_cou.apply(round_up_to_nearest_half_int)
delivery_x=compare['Zone_y'].str.upper()
delivery_cou=compare['Zone_x'].str.upper()

In [15]:
data1={'Order ID':orderid,'AWB Number':awb,'Total weight as per X (KG)':total_wt,'Weight slab as per X (KG)':wt_slab_x,'Total weight as per Courier Company (KG)':toatl_wt_cou,'Weight slab charged by Courier Company (KG)':wt_slab_cou,'Delivery Zone as per X':delivery_x,'Delivery Zone charged by Courier Company':delivery_cou}

In [16]:
res=pd.DataFrame(data=data1)
res

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,B,B
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,D,D
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,D,D
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B
...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,B,D
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,B,D
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,D,D
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,D,D


**Excepted charges computation**

In [17]:
excepted_charges=[]

for i in range(len(compare)):

  fwd_category='fwd_'+compare['Zone_y'][i]
  fwd_fixed=cou_rates[fwd_category+'_fixed'][0]
  fwd_additional=cou_rates[fwd_category+'_additional'][0]
  rto_category='rto_'+compare['Zone_y'][i]
  rto_fixed=cou_rates[rto_category+'_fixed'][0]
  rto_additional=cou_rates[rto_category+'_additional'][0]
  if compare['Type of Shipment'][i]=='Forward charges':
    if res['Weight slab as per X (KG)'][i]<=0.5:
      excepted_charges.append(fwd_fixed)
    else:
      additional_weight=(res['Weight slab as per X (KG)'][i]-0.5)/0.5
      excepted_charges.append(fwd_fixed+additional_weight*fwd_additional)

  if compare['Type of Shipment'][i]=='Forward and RTO charges':
     if res['Weight slab as per X (KG)'][i]<=0.5:
       excepted_charges.append(fwd_fixed+rto_fixed)
     else:
       additional_weight=(res['Weight slab as per X (KG)'][i]-0.5)/0.5
       excepted_charges.append(fwd_fixed+additional_weight*(fwd_additional+rto_additional))

#print(excepted_charges)
res['Expected Charge as per X (Rs.)']=excepted_charges





In [18]:
res['Charges Billed by Courier Company (Rs.)']=compare['Billing Amount (Rs.)']

In [19]:
res['Difference Between Expected Charges and Billed Charges (Rs.)']=res['Expected Charge as per X (Rs.)']-res['Charges Billed by Courier Company (Rs.)']

## **Final Output result**

In [20]:
res

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.220,0.5,2.92,3.0,B,B,33.0,174.5,-141.5
1,2001806226,1091117222065,0.480,0.5,0.68,1.0,D,D,45.4,90.2,-44.8
2,2001806229,1091117222080,0.500,0.5,0.71,1.0,D,D,45.4,90.2,-44.8
3,2001806232,1091117222124,1.302,1.5,1.30,1.5,D,D,135.0,135.0,0.0
4,2001806233,1091117222135,0.245,0.5,0.78,1.0,B,B,33.0,61.3,-28.3
...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,1091121183730,0.477,0.5,0.50,0.5,B,D,33.0,45.4,-12.4
120,2001822466,1091121305541,1.376,1.5,1.10,1.5,B,D,89.6,135.0,-45.4
121,2001823564,1091121666133,0.672,1.0,0.70,1.0,D,D,135.0,172.8,-37.8
122,2001825261,1091121981575,1.557,2.0,1.60,2.0,D,D,314.2,345.0,-30.8


In [21]:
res[res['Order ID']==2001806210]

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.5,2.92,3.0,B,B,33.0,174.5,-141.5


## **Summary Table**

In [22]:
correctly_charged=res[res['Difference Between Expected Charges and Billed Charges (Rs.)'] ==0]

a=len(correctly_charged)
b=correctly_charged['Charges Billed by Courier Company (Rs.)'].sum()
correct=['Total orders where X has been correctly charged',a,b]
correct
                  

['Total orders where X has been correctly charged', 16, 1268.6]

In [23]:
over_charged=res[res['Difference Between Expected Charges and Billed Charges (Rs.)']<0]
c=len(over_charged)
d=np.absolute(over_charged['Charges Billed by Courier Company (Rs.)']).sum()
over=['Total Orders where X has been overcharged',c,d]
over

['Total Orders where X has been overcharged', 95, 11114.5]

In [24]:
under_charged=res[res['Difference Between Expected Charges and Billed Charges (Rs.)']>0]
e=len(under_charged)
f=np.absolute(sum(under_charged['Charges Billed by Courier Company (Rs.)']))
under=['Total Orders where X has been overcharged',e,f]
under

['Total Orders where X has been overcharged', 13, 1265.1000000000001]

In [25]:
res2=pd.DataFrame([correct,over,under],columns=['Descprition','Count','Amount (RS)'])
res2

,Descprition,Count,Amount (RS)
0,Total orders where X has been correctly charged,16,1268.6
1,Total Orders where X has been overcharged,95,11114.5
2,Total Orders where X has been overcharged,13,1265.1


In [26]:
with pd.ExcelWriter('Cointab.xlsx') as writer:
  res.to_excel(writer,sheet_name='Assignment')
  res2.to_excel(writer,sheet_name='Summary')